### Imports and Setup

In [1]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

#importing models
from models.dt_model import dt_model
from models.rf_model import rf_model
from models.gb_model import gb_model
from models.knn_model import knn_model
from models.lr_model import lr_model
from models.lr_l1_model import lr_l1_model
from models.lr_l2_model import lr_l2_model
from models.lr_enet_model import lr_enet_model
from models.gnb_model import gnb_model
from models.lda_model import lda_model
from models.svm_linear_model import svm_linear_model
from models.svm_non_linear_model import svm_non_linear_model

#choose seed for run
seed = 123456
#set number of training runs desired
runs = 10

### Load and Explore the Data

In [2]:
#Loading wine dataset as proof of concept
df = pd.read_csv("./data/wine.csv")
target_column = "Wine"
X = df.drop(columns=[target_column])
y = df[target_column]
df.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


### Initializing Models and Defining Result Arrays

In [3]:
#define helper models
dt = dt_model()
rf = rf_model()
gb = gb_model()
knn = knn_model()
lr = lr_model()
lr_l1 = lr_l1_model()
lr_l2 = lr_l2_model()
lr_enet = lr_enet_model()
gnb = gnb_model()
lda = lda_model()
svm_linear = svm_linear_model()
svm_non_linear = svm_non_linear_model()

#define results arrays
dtEvalResults = np.zeros((runs, 5))
rfEvalResults = np.zeros((runs,5))
gbEvalResults = np.zeros((runs, 5))
knnEvalResults = np.zeros((runs, 5))
lrEvalResults = np.zeros((runs, 5))
lrL1EvalResults = np.zeros((runs, 5))
lrL2EvalResults = np.zeros((runs, 5))
lrEnetEvalResults = np.zeros((runs, 5))
gnbEvalResults = np.zeros((runs, 5))
ldaEvalResults = np.zeros((runs, 5))
svmLinearEvalResults = np.zeros((runs, 5))
svmNonLinearEvalResults = np.zeros((runs, 5))

### Training Loop

In [4]:
for r in range(runs):
    #first, split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

    #next, preprocess data
    #TODO: HANDLE DIFFERENT SCALING
    #TODO: Variance Threshold

    le = LabelEncoder()
    y_train = le.fit_transform(y_train)
    y_test = le.transform(y_test)

    scaler = MinMaxScaler()
    X_train_scaled_standard = scaler.fit_transform(X_train)
    X_test_scaled_standard = scaler.transform(X_test)

    # Training the models and storing the evaluation results
    # decision tree
    dt.train(X_train_scaled_standard, y_train)
    dtEvalResults[r, :] = dt.predict(X_test_scaled_standard, y_test)
    
    # random forest
    rf.train(X_train_scaled_standard, y_train)
    rfEvalResults[r, :] = rf.predict(X_test_scaled_standard, y_test)
    
    # gradient boosting
    gb.train(X_train_scaled_standard, y_train)
    gbEvalResults[r, :] = gb.predict(X_test_scaled_standard, y_test)
    
    # k-nearest neighbors
    knn.train(X_train_scaled_standard, y_train)
    knnEvalResults[r, :] = knn.predict(X_test_scaled_standard, y_test)
    
    # logistic regression
    lr.train(X_train_scaled_standard, y_train)
    lrEvalResults[r, :] = lr.predict(X_test_scaled_standard, y_test)
    
    # logistic regression - L1 (Lasso)
    lr_l1.train(X_train_scaled_standard, y_train)
    lrL1EvalResults[r, :] = lr_l1.predict(X_test_scaled_standard, y_test)

    # logistic regression - L2 (Ridge)
    lr_l2.train(X_train_scaled_standard, y_train)
    lrL2EvalResults[r, :] = lr_l2.predict(X_test_scaled_standard, y_test)

    # logistic regression - Elastic Net
    lr_enet.train(X_train_scaled_standard, y_train)
    lrEnetEvalResults[r, :] = lr_enet.predict(X_test_scaled_standard, y_test)
    
    # gaussian naive bayes
    gnb.train(X_train_scaled_standard, y_train)
    gnbEvalResults[r, :] = gnb.predict(X_test_scaled_standard, y_test)
    
    # linear discriminant analysis
    lda.train(X_train_scaled_standard, y_train)
    ldaEvalResults[r, :] = lda.predict(X_test_scaled_standard, y_test)
    
    # linear support vector machine
    svm_linear.train(X_train_scaled_standard, y_train)
    svmLinearEvalResults[r, :] = svm_linear.predict(X_test_scaled_standard, y_test)
    
    # non-linear support vector machine
    svm_non_linear.train(X_train_scaled_standard, y_train)
    svmNonLinearEvalResults[r, :] = svm_non_linear.predict(X_test_scaled_standard, y_test)


### Computing & Printing Average Metrics Functions

In [5]:
def average_metrics(eval_results):
    avg_accuracy = np.mean(eval_results[:, 0])
    avg_precision = np.mean(eval_results[:, 1])
    avg_recall = np.mean(eval_results[:, 2])
    avg_f1 = np.mean(eval_results[:, 3])
    avg_auc = np.nanmean(eval_results[:, 4])
    
    return avg_accuracy, avg_precision, avg_recall, avg_f1, avg_auc


def print_model_results(model_name, avg_accuracy, avg_precision, avg_recall, avg_f1, avg_auc):
    print(f"{model_name} Model:")
    print(f"Average Accuracy: {avg_accuracy}")
    print(f"Average Precision: {avg_precision}")
    print(f"Average Recall: {avg_recall}")
    print(f"Average F1: {avg_f1}")
    print(f"Average AUC: {avg_auc if avg_auc is not None else 'N/A'}")
    print("\n")

### Displaying Average Metrics

In [6]:
# Computing average metrics
dt_avg_accuracy, dt_avg_precision, dt_avg_recall, dt_avg_f1, dt_avg_auc = average_metrics(dtEvalResults)
rf_avg_accuracy, rf_avg_precision, rf_avg_recall, rf_avg_f1, rf_avg_auc = average_metrics(rfEvalResults)
gb_avg_accuracy, gb_avg_precision, gb_avg_recall, gb_avg_f1, gb_avg_auc = average_metrics(gbEvalResults)
knn_avg_accuracy, knn_avg_precision, knn_avg_recall, knn_avg_f1, knn_avg_auc = average_metrics(knnEvalResults)
lr_avg_accuracy, lr_avg_precision, lr_avg_recall, lr_avg_f1, lr_avg_auc = average_metrics(lrEvalResults)
lr_l1_avg_accuracy, lr_l1_avg_precision, lr_l1_avg_recall, lr_l1_avg_f1, lr_l1_avg_auc = average_metrics(lrL1EvalResults)
lr_l2_avg_accuracy, lr_l2_avg_precision, lr_l2_avg_recall, lr_l2_avg_f1, lr_l2_avg_auc = average_metrics(lrL2EvalResults)
lr_enet_avg_accuracy, lr_enet_avg_precision, lr_enet_avg_recall, lr_enet_avg_f1, lr_enet_avg_auc = average_metrics(lrEnetEvalResults)
gnb_avg_accuracy, gnb_avg_precision, gnb_avg_recall, gnb_avg_f1, gnb_avg_auc = average_metrics(gnbEvalResults)
lda_avg_accuracy, lda_avg_precision, lda_avg_recall, lda_avg_f1, lda_avg_auc = average_metrics(ldaEvalResults)
svm_linear_avg_accuracy, svm_linear_avg_precision, svm_linear_avg_recall, svm_linear_avg_f1, svm_linear_avg_auc = average_metrics(svmLinearEvalResults)
svm_non_linear_avg_accuracy, svm_non_linear_avg_precision, svm_non_linear_avg_recall, svm_non_linear_avg_f1, svm_non_linear_avg_auc = average_metrics(svmNonLinearEvalResults)

# Printing
print_model_results("Decision Tree", dt_avg_accuracy, dt_avg_precision, dt_avg_recall, dt_avg_f1, dt_avg_auc)
print_model_results("Random Forest", rf_avg_accuracy, rf_avg_precision, rf_avg_recall, rf_avg_f1, rf_avg_auc)
print_model_results("Gradient Boosting", gb_avg_accuracy, gb_avg_precision, gb_avg_recall, gb_avg_f1, gb_avg_auc)
print_model_results("K-Nearest Neighbors", knn_avg_accuracy, knn_avg_precision, knn_avg_recall, knn_avg_f1, knn_avg_auc)
print_model_results("Logistic Regression", lr_avg_accuracy, lr_avg_precision, lr_avg_recall, lr_avg_f1, lr_avg_auc)
print_model_results("Logistic Regression - Lasso (L1)", lr_l1_avg_accuracy, lr_l1_avg_precision, lr_l1_avg_recall, lr_l1_avg_f1, lr_l1_avg_auc)
print_model_results("Logistic Regression - Ridge (L2)", lr_l2_avg_accuracy, lr_l2_avg_precision, lr_l2_avg_recall, lr_l2_avg_f1, lr_l2_avg_auc)
print_model_results("Logistic Regression - Elastic Net", lr_enet_avg_accuracy, lr_enet_avg_precision, lr_enet_avg_recall, lr_enet_avg_f1, lr_enet_avg_auc)
print_model_results("Gaussian Naive Bayes", gnb_avg_accuracy, gnb_avg_precision, gnb_avg_recall, gnb_avg_f1, gnb_avg_auc)
print_model_results("Linear Discriminant Analysis", lda_avg_accuracy, lda_avg_precision, lda_avg_recall, lda_avg_f1, lda_avg_auc)
print_model_results("Linear SVM", svm_linear_avg_accuracy, svm_linear_avg_precision, svm_linear_avg_recall, svm_linear_avg_f1, svm_linear_avg_auc)
print_model_results("Non-linear SVM", svm_non_linear_avg_accuracy, svm_non_linear_avg_precision, svm_non_linear_avg_recall, svm_non_linear_avg_f1, svm_non_linear_avg_auc)

Decision Tree Model:
Average Accuracy: 0.886013986013986
Average Precision: 0.8926619201437227
Average Recall: 0.886013986013986
Average F1: 0.8851216251211429
Average AUC: 0.9137060503834954


Random Forest Model:
Average Accuracy: 0.9251748251748252
Average Precision: 0.9319687092870705
Average Recall: 0.9251748251748252
Average F1: 0.924771118269278
Average AUC: 0.9910076361154692


Gradient Boosting Model:
Average Accuracy: 0.8825174825174825
Average Precision: 0.9025513695109237
Average Recall: 0.8825174825174825
Average F1: 0.8797562521900723
Average AUC: 0.9772061903249037


K-Nearest Neighbors Model:
Average Accuracy: 0.9475524475524475
Average Precision: 0.951894097383485
Average Recall: 0.9475524475524475
Average F1: 0.9469731197840879
Average AUC: 0.9926022095163187


Logistic Regression Model:
Average Accuracy: 0.9433566433566434
Average Precision: 0.9500753615712819
Average Recall: 0.9433566433566434
Average F1: 0.9431950106972524
Average AUC: 0.9936743909498436


Logistic